In [ ]:
# Plot options
import matplotlib.pyplot as plt

# Import required libraries
import numpy as np
import os

# Import karabo libraries
import karabo_bridge as kb
import karabo_data as kd
import xfelmay2019 as xfel


# Plot maximum scattering image

In [ ]:

def analyzeMaximumImage( runNumber, 
                        path='/gpfs/exfel/exp/SQS/201921/p002430/raw',
                        maskRadius=100,
                        saveDir='/home/mrware/Photos'):
    '''
    finds maximum scattering and plots that image, saving to saveDir
    input:
        runNumber: to analyze
        path: to experiment H5s
        maskRadius: radius below which to ignore for determining max scattering
        saveDir: path to save picture to
    '''
    path='/gpfs/exfel/exp/SQS/201921/p002430/raw'
    run = xfel.runFormat( runNumber )
    runData = kd.RunDirectory(path+run)
    trainIds = runData.train_ids
    
    scatImages = np.asarray(runData.get_array('SQS_DPU_LIC/CAM/YAG_UPSTR:daqOutput', 'data.image.pixels'))

    NT,NX,NY = scatImages.shape
    xs = np.arange(NX)
    ys = np.arange(NY)
    YY,XX = np.meshgrid( ys , xs )
    RR = np.sqrt( ( XX - NX/2. )**2 + ( YY - NY/2. )**2 )
    MASK = RR < maskRadius
    
    maskedScats = np.copy(scatImages)
    maskedScats[:,MASK] = 0.
    
    scaledScat = np.sum( maskedScats, (1,2) ) 
    
    indMax = np.argmax(scaledScat)
    indMin = np.argmin(scaledScat)

    scatMax = scatImages[indMax,:,:].astype(float)
    scatMin = scatImages[indMin, :, :].astype(float)
    
    plt.figure(5)
    plt.imshow(scatMax)
    plt.title( 'Max scattering from run %d' % runNumber )
    plt.colorbar()
    plt.show()
    plt.pause(2)
    plt.savefig(saveDir + '/scatmax-run-%d' % runNumber )
    
    plt.figure(5)
    plt.imshow( np.log(np.abs(scatMax/np.sum(scatMax.flatten()) -   scatMin/np.sum(scatMin.flatten())))  )
    plt.title( 'Log of difference between scatMax/intensity-scatMin/intensity from run %d' % runNumber )
    plt.colorbar()
    plt.show()
    plt.pause(2)
    plt.savefig( saveDir + '/scatdiff-run-%d' % runNumber )
    
    del scatImages
    del maskedScats


In [ ]:
analyzeMaximumImage( runNumber=247, 
                        path='/gpfs/exfel/exp/SQS/201921/p002430/raw',
                        maskRadius=200,
                        saveDir = '/home/mrware')

# Plot set of nlargest scattering images

In [ ]:

def analyzeMaximumImages( runNumber, 
                        path='/gpfs/exfel/exp/SQS/201921/p002430/raw',
                        maskRadius=100,
                        saveDir='/home/mrware/Photos',
                        nlargest=10):
    '''
    finds maximum scattering images and plots that image, saving to saveDir
    input:
        runNumber: to analyze
        path: to experiment H5s
        maskRadius: radius below which to ignore for determining max scattering
        saveDir: path to save picture to
        nlargest: number of largest images to plot
    '''
    path='/gpfs/exfel/exp/SQS/201921/p002430/raw'
    run = xfel.runFormat( runNumber )
    runData = kd.RunDirectory(path+run)
    trainIds = runData.train_ids
    
    scatImages = np.asarray(runData.get_array('SQS_DPU_LIC/CAM/YAG_UPSTR:daqOutput', 'data.image.pixels'))

    NT,NX,NY = scatImages.shape
    xs = np.arange(NX)
    ys = np.arange(NY)
    YY,XX = np.meshgrid( ys , xs )
    RR = np.sqrt( ( XX - NX/2. )**2 + ( YY - NY/2. )**2 )
    MASK = RR < maskRadius
    
    maskedScats = np.copy(scatImages)
    maskedScats[:,MASK] = 0.
    
    scaledScat = np.sum( maskedScats, (1,2) ) 
    indMaxToMin = np.argsort( scaledScat )[-nlargest:]
    
    fig, axes = plt.subplots(nlargest, 1, figsize=(10,100))
    
    for idx,ax in enumerate(axes):
        realInd = indMaxToMin[idx]
        plotme=scatImages[realInd,:,:].squeeze()
        im=ax.imshow( plotme.astype(float) )
        ax.set_title('run %d and tid %d' % (runNumber, trainIds[realInd]))

    plt.show()
    plt.pause(2)
    plt.savefig(saveDir + '/scatmaxes-run-%d' % runNumber )
    
    
    del scatImages
    del maskedScats


In [ ]:
%matplotlib inline
analyzeMaximumImages( runNumber=244, 
                        path='/gpfs/exfel/exp/SQS/201921/p002430/raw',
                        maskRadius=200,
                        saveDir = '/home/mrware')